In [1]:
import Model_Blocks.Transformer as tfm
import Data_handler as dh
import Training as tr
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
import torch
import os

In [2]:
# Checking if Dataloarder is already saved 
if os.path.exists('train_loader.pt') and os.path.exists('test_loader.pt'):
    train_loader = torch.load('train_loader.pt')
    test_loader = torch.load('test_loader.pt')
else:
    df, vec_en, vec_fr, max_len_en, max_len_fr = dh.prepare_data()
    padding_fr = vec_fr.vocabulary_['PPPading']
    padding_en = vec_en.vocabulary_['PPPading']
    len_fr_vocab = len(vec_fr.vocabulary_)
    len_en_vocab = len(vec_en.vocabulary_)

    train_dataset,test_dataset = train_test_split(df, test_size=0.16, random_state=23)

    train_dataset = dh.VocabDataset(train_dataset, padding_fr, padding_en)
    test_dataset = dh.VocabDataset(test_dataset, padding_fr, padding_en)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
    torch.save(train_loader, 'train_loader.pt')
    torch.save(test_loader, 'test_loader.pt')


In [3]:
model = tfm.Transformer(len_en_vocab, len_fr_vocab)

In [ ]:
tr.train_model(model, train_loader, test_loader, optimizer=torch.optim.Adam(model.parameters()), criterion= torch.nn.CrossEntropyLoss(), padding_fr=padding_fr, padding_en=padding_en)